In [1]:
import numpy as np
import umap
import plotly.express as px
from openTSNE import TSNE

from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
# feature_2d = umap.UMAP().fit_transform(feature)
def load_data(classifier, data_type, noise, source, target, othertxt=''):
    p = '/Volumes/ZelinDisk/big/San_data/'
    path_feature = f'{p}/{classifier}/{data_type}/noise_feature_test_{noise}{othertxt}_{source}_{target}.npy'
    path_label = f'{p}/{classifier}/{data_type}/noise_label_all_{noise}{othertxt}_{source}_{target}.npy'
    path_pre = f'{p}/{classifier}/{data_type}/noise_pred_all_{noise}{othertxt}_{source}_{target}.npy'
    feature = np.load(path_feature)
    label = [int(l) for l in np.load(path_label).tolist()]
    pre = [int(l) for l in np.load(path_pre).tolist()]

    # print(feature.shape)
    # print(len(label))
    # print(len(pre))
    return feature, label, pre

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 20}

In [45]:
fig = make_subplots(rows=2, cols=3)


# source
noise = 0.3
classifier = 'OVANet'
data_type = 'source'
source = 'amazon'
target = 'dslr'
feature_s, label_s, pre_s = load_data(classifier, data_type, noise, source, target)

# for noise in [0.1,0.2,0.3,0.4,0.5,0.6]:
for noise in [0.3]:
    # noise = 0.3
    classifier = 'OVANet'
    data_type = 'target'
    # source = 'amazon'
    # source = 'dslr'
    source = 'webcam'
    target = 'dslr'
    # target = 'amazon'
    # target = 'webcam'
    
    feature_t, label_t, pre_t = load_data(classifier, data_type, noise, source, target)
    label_t = np.array(label_t)
    pre_t = np.array(pre_t)
    mask_nuknown = np.array(label_t)==20
    acc_ova = sum( label_t[mask_nuknown]==pre_t[mask_nuknown] )/len(label_t[mask_nuknown])
    print(f'noise {float(noise)} accuracy source:{acc_ova}')

    # noise = 0.1
    classifier = 'SAN'
    # data_type = 'target'
    # source = 'amazon'
    # target = 'dslr'
    acc_list = []
    for j1 in [0.5]:
        for j2 in [0.5, 1.5]:
            for j3 in [0.4, 0.5, 0.6]:
                for j4 in [0.1, 0.2]:
                    try:
                        feature_t, label_t, pre_t = load_data(classifier, data_type, noise, source, target, othertxt=f'_{j1}_{j2}_{j3}_{j4}')
                        label_t = np.array(label_t)
                        mask_nuknown = np.array(label_t)==20
                        pre_t = np.array(pre_t)
                        acc_list.append(sum( label_t[mask_nuknown]==pre_t[mask_nuknown] )/len(label_t[mask_nuknown]))
                        print(f'_{j1}_{j2}_{j3}_{j4}', sum( label_t[mask_nuknown]==pre_t[mask_nuknown] )/len(label_t[mask_nuknown]))
                    except:
                        pass
                    # feature_t, label_t, pre_t = load_data(classifier, data_type, noise, source, target, othertxt='_0.5_1.0_0.4_0.2')
    print(f'san best acc {max(acc_list)}, len {len(acc_list)}')
    print(f'acc gap {max(acc_list)-acc_ova}')
    # calculate the accuracy


    

noise 0.3 accuracy source:0.8685714285714285
_0.5_1.5_0.4_0.1 0.9714285714285714
_0.5_1.5_0.4_0.2 0.9542857142857143
_0.5_1.5_0.5_0.1 1.0
_0.5_1.5_0.5_0.2 0.9942857142857143
_0.5_1.5_0.6_0.1 0.96
_0.5_1.5_0.6_0.2 0.9542857142857143
san best acc 1.0, len 6
acc gap 0.13142857142857145


In [53]:

fig = make_subplots(rows=2, cols=3)


# source
noise = 0.4
classifier = 'OVANet'
data_type = 'source'
source = 'webcam'
target = 'dslr'
feature_s, label_s, pre_s = load_data(classifier, data_type, noise, source, target)
musk_10_s = ((np.array(label_s)>=15).astype(np.float32) +(np.array(label_s).astype(np.float32)<5))>0
# print(musk_10_s)
feature_s = feature_s[musk_10_s]
label_s = np.array(label_s)[musk_10_s]
pre_s = np.array(pre_s)[musk_10_s]

noise = 0.4
classifier = 'OVANet'
data_type = 'target'
source = 'webcam'
target = 'dslr'
feature_t, label_t, pre_t = load_data(classifier, data_type, noise, source, target)
musk_10_t = ((np.array(label_t)>=15)+(np.array(label_t)<5))>0
feature_t = feature_t[musk_10_t]
label_t = np.array(label_t)[musk_10_t]
pre_t = np.array(pre_t)[musk_10_t]

# calculate the accuracy
print(f'accuracy source: {sum(np.array(label_t)==np.array(pre_t))/len(label_t)}')

feature_cat_2d = TSNE(
    perplexity=40,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    ).fit(np.concatenate([feature_s, feature_t]))

feature_2d_s = feature_cat_2d[:feature_s.shape[0],:]
feature_2d_t = feature_cat_2d[feature_s.shape[0]:,:]

fig.add_trace(
    go.Scatter(
        x=feature_2d_s[:,0], y=feature_2d_s[:,1], mode='markers',
        marker=dict(color=label_s), text=label_s
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=feature_2d_s[:,0], y=feature_2d_s[:,1], mode='markers',
        marker=dict(color=pre_s), text=pre_s
        ),
    row=1, col=2
)

deta = (np.array(label_s)-np.array(pre_s)).tolist()
deta_color = ['black' if d == 0 else 'red' for d in deta]
fig.add_trace(
    go.Scatter(
        x=feature_2d_s[:,0], y=feature_2d_s[:,1], mode='markers',
        marker=dict(color=deta_color),
        ),
    row=1, col=3
)

# target

fig.add_trace(
    go.Scatter(
        x=feature_2d_t[:,0], y=feature_2d_t[:,1], mode='markers',
        marker=dict(color=label_t, ), text=label_t
    ),
    row=2, col=1,
)
fig.add_trace(
    go.Scatter(
        x=feature_2d_t[:,0], y=feature_2d_t[:,1], mode='markers',
        marker=dict(color=pre_t,), text=pre_t
        ),
    row=2, col=2
)

deta = (np.array(label_t)-np.array(pre_t)).tolist()
deta_color=[]
for d in deta:
    if d == 0:
        deta_color.append('black')
    if d >0:
        deta_color.append('red')
    if d <0:
        deta_color.append('green')
# deta_color = ['black' if d == 0 elif d>0 'red' elif d>0 'red' for d in deta]
fig.add_trace(
    go.Scatter(
        x=feature_2d_t[:,0], y=feature_2d_t[:,1], mode='markers',
        marker=dict(color=deta_color,), 
        ),
    row=2, col=3
)


fig.update_layout(width=1500, height=1200, title=f'classifier:{classifier} source:{source} target:{target} noise:{noise} data_type:{data_type}')
fig.show()

accuracy source: 0.8148148148148148
--------------------------------------------------------------------------------
TSNE(n_jobs=8, perplexity=40, random_state=42, verbose=True)
--------------------------------------------------------------------------------
===> Finding 120 nearest neighbors using exact search using euclidean distance...
   --> Time elapsed: 0.10 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.03 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 2.28 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 2.3800, 50 iterations in 0.1839 sec
Iteration  100, KL divergence 2.3789, 50 iterations in 0.1803 sec
Iteration  150, KL divergence 2.3271, 50 iterations in 0.1178 sec
Iteration  200, KL divergence 2.3197, 50 iterations in 0.1020 sec
Iteration  250, KL divergence 2.3212, 50 iterations in 0.0755 sec
   --> Time elapsed: 0.66 seconds
===> Running optimization wit

In [52]:

fig = make_subplots(rows=2, cols=3)


# source
noise = 0.3
classifier = 'SAN'
data_type = 'source'
source = 'webcam'
target = 'dslr'
feature_s, label_s, pre_s = load_data(classifier, data_type, noise, source, target, othertxt='_0.5_1.5_0.5_0.1')

musk_10_s = ((np.array(label_s)>15).astype(np.float32) +(np.array(label_s).astype(np.float32)<5))>0
# print(musk_10_s)
feature_s = feature_s[musk_10_s]
label_s = np.array(label_s)[musk_10_s]
pre_s = np.array(pre_s)[musk_10_s]


noise = 0.3
classifier = 'SAN'
data_type = 'target'
source = 'webcam'
target = 'dslr'
feature_t, label_t, pre_t = load_data(classifier, data_type, noise, source, target, othertxt='_0.5_1.5_0.5_0.1')
musk_10_t = ((np.array(label_t)>15)+(np.array(label_t)<5))>0
feature_t = feature_t[musk_10_t]
label_t = np.array(label_t)[musk_10_t]
pre_t = np.array(pre_t)[musk_10_t]

feature_cat_2d = TSNE(
    perplexity=50,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
    ).fit(np.concatenate([feature_s, feature_t]))

feature_2d_s = feature_cat_2d[:feature_s.shape[0],:]
feature_2d_t = feature_cat_2d[feature_s.shape[0]:,:]

fig.add_trace(
    go.Scatter(
        x=feature_2d_s[:,0], y=feature_2d_s[:,1], mode='markers',
        marker=dict(color=label_s), text=label_s
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=feature_2d_s[:,0], y=feature_2d_s[:,1], mode='markers',
        marker=dict(color=pre_s), text=pre_s
        ),
    row=1, col=2
)

deta = (np.array(label_s)-np.array(pre_s)).tolist()
deta_color = ['black' if d == 0 else 'red' for d in deta]
fig.add_trace(
    go.Scatter(
        x=feature_2d_s[:,0], y=feature_2d_s[:,1], mode='markers',
        marker=dict(color=deta_color),
        ),
    row=1, col=3
)

# target

fig.add_trace(
    go.Scatter(
        x=feature_2d_t[:,0], y=feature_2d_t[:,1], mode='markers',
        marker=dict(color=label_t), text=label_t
    ),
    row=2, col=1,
)
fig.add_trace(
    go.Scatter(
        x=feature_2d_t[:,0], y=feature_2d_t[:,1], mode='markers',
        marker=dict(color=pre_t), text=pre_t
        ),
    row=2, col=2
)

deta = (np.array(label_t)-np.array(pre_t)).tolist()
deta_color=[]
for d in deta:
    if d == 0:
        deta_color.append('black')
    if d >0:
        deta_color.append('red')
    if d <0:
        deta_color.append('green')
# deta_color = ['black' if d == 0 elif d>0 'red' elif d>0 'red' for d in deta]
fig.add_trace(
    go.Scatter(
        x=feature_2d_t[:,0], y=feature_2d_t[:,1], mode='markers',
        marker=dict(color=deta_color),
        ),
    row=2, col=3
)

acc = label_t==pre_t
fig.update_layout(
    width=1500,
    height=1200,
    title=f'classifier:{classifier} source:{source} target:{target} noise:{noise} data_type:{data_type} acc:{acc}'
    )
fig.show()

--------------------------------------------------------------------------------
TSNE(n_jobs=8, perplexity=50, random_state=42, verbose=True)
--------------------------------------------------------------------------------
===> Finding 150 nearest neighbors using exact search using euclidean distance...
   --> Time elapsed: 0.20 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.03 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 1.77 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 2.0687, 50 iterations in 0.1865 sec
Iteration  100, KL divergence 1.9893, 50 iterations in 0.1357 sec
Iteration  150, KL divergence 2.0294, 50 iterations in 0.1034 sec
Iteration  200, KL divergence 2.0366, 50 iterations in 0.0910 sec
Iteration  250, KL divergence 2.0503, 50 iterations in 0.0997 sec
   --> Time elapsed: 0.62 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 5

In [ ]:
fig1

In [ ]:
(np.array(label_s)==np.array(pre_s)).sum()

In [ ]:
(np.array(label_t)==np.array(pre_t)).sum()
